<a href="https://www.kaggle.com/code/minanalminazor/heart-failure-classification-with-dl-tensorflow?scriptVersionId=147222718" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv


In [2]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer
from sklearn.metrics import classification_report
from tensorflow.keras.utils import to_categorical

In [3]:
df = pd.read_csv("/kaggle/input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv")
print(df.info())
print('Classes and number of values in the dataset',Counter(df['DEATH_EVENT']))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   age                       299 non-null    float64
 1   anaemia                   299 non-null    int64  
 2   creatinine_phosphokinase  299 non-null    int64  
 3   diabetes                  299 non-null    int64  
 4   ejection_fraction         299 non-null    int64  
 5   high_blood_pressure       299 non-null    int64  
 6   platelets                 299 non-null    float64
 7   serum_creatinine          299 non-null    float64
 8   serum_sodium              299 non-null    int64  
 9   sex                       299 non-null    int64  
 10  smoking                   299 non-null    int64  
 11  time                      299 non-null    int64  
 12  DEATH_EVENT               299 non-null    int64  
dtypes: float64(3), int64(10)
memory usage: 30.5 KB
None
Classes and n

In [4]:
y = df["DEATH_EVENT"]
x = df[['age','anaemia','creatinine_phosphokinase','diabetes','ejection_fraction','high_blood_pressure','platelets','serum_creatinine','serum_sodium','sex','smoking','time']]

x = pd.get_dummies(x)

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(x,y,test_size = 0.3, random_state = 0)

ct = ColumnTransformer([("numeric", StandardScaler(), ['age','creatinine_phosphokinase','ejection_fraction','platelets','serum_creatinine','serum_sodium','time'])])
X_train = ct.fit_transform(X_train)
X_test = ct.transform(X_test)

le = LabelEncoder()
Y_train = le.fit_transform(Y_train.astype(str))
Y_test = le.transform(Y_test.astype(str))

Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)

In [6]:
model = Sequential()
model.add(InputLayer(input_shape=(X_train.shape[1],)))
model.add(Dense(12,activation='relu'))
model.add(Dense(2,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs = 100, batch_size = 16, verbose = 1)

loss, acc = model.evaluate(X_train, Y_train, verbose=0)
print("Loss", loss, "Accuracy:", acc)

Epoch 1/100
14/14 [==============================] - 1s 2ms/step - loss: 0.8544 - accuracy: 0.3589
Epoch 2/100
14/14 [==============================] - 0s 2ms/step - loss: 0.8169 - accuracy: 0.3828
Epoch 3/100
14/14 [==============================] - 0s 2ms/step - loss: 0.7834 - accuracy: 0.4211
Epoch 4/100
14/14 [==============================] - 0s 2ms/step - loss: 0.7547 - accuracy: 0.4785
Epoch 5/100
14/14 [==============================] - 0s 2ms/step - loss: 0.7273 - accuracy: 0.5646
Epoch 6/100
14/14 [==============================] - 0s 2ms/step - loss: 0.7056 - accuracy: 0.6459
Epoch 7/100
14/14 [==============================] - 0s 1ms/step - loss: 0.6849 - accuracy: 0.6842
Epoch 8/100
14/14 [==============================] - 0s 2ms/step - loss: 0.6651 - accuracy: 0.6938
Epoch 9/100
14/14 [==============================] - 0s 2ms/step - loss: 0.6457 - accuracy: 0.7177
Epoch 10/100
14/14 [==============================] - 0s 2ms/step - loss: 0.6300 - accuracy: 0.7177
Epoch 11/

In [7]:
y_estimate = model.predict(X_test,verbose=0)
y_estimate = np.argmax(y_estimate,axis = 1)
y_true = np.argmax(Y_test,axis=1)

print(classification_report(y_true, y_estimate))

              precision    recall  f1-score   support

           0       0.82      0.95      0.88        62
           1       0.83      0.54      0.65        28

    accuracy                           0.82        90
   macro avg       0.83      0.74      0.77        90
weighted avg       0.82      0.82      0.81        90

